In [1]:
from typing import Optional, Tuple

In [2]:
import torch

In [3]:
import torch.nn as nn

In [4]:
class SiglipVisionConfig:
    
    def __init__(self,
                hidden_size=768,
                intermediate_size=3072,
                num_hidden_layers=12,
                num_attention_heads=12,
                num_channels=3,
                image_size=224,
                patch_size=16,
                layer_norm_eps=1e-6,
                attention_dropout=0.0,
                num_image_tokens: int=None,
                **kwargs
                ):
        super().__init__()

        self.hidden_size = hidden_size
        self.intermediate_size = intermediate_size
        self.num_hidden_layers = num_hidden_layers
        self.num_attention_heads = num_attention_heads
        self.num_channels = num_channels
        self.patch_size = patch_size
        self.image_size = image_size
        self.attention_dropout = attention_dropout
        self.layer_norm_eps = layer_norm_eps
        self.num_image_tokens = num_image_tokens

In [8]:
class SiglipVisionEmbedding(nn.Embedding):
    
    
    def __init__(self,config: SiglipVisionConfig):
        
        super().__init__()
        self.config = config
        self.embed_dim = config.hidden_size
        self.image_size = config.image_size
        self.patch_size = config.patch_size
        
        
        self.patch_embedding = nn.Conv2d(
            in_channels=config.num_channels,
            out_channels=self.embed_dim,
            kernel_size=self.patch_size,
            stride=self.patch_size,
            padding='valid',
        )
        
        self.num_patches = (self.image_size // self.patch_size) ** 2
        self.num_positions = self.num_patches
        self.position_embedding = nn.Embedding(self.num_positions,self.embed_dim)
        
        self.register_buffer(
          'position_ids',
          torch.arange(self.num_positions).expand((1,-1)),
          persistent=False
        )
    
    def forward(self,pixel_values: torch.FloatTensor) -> torch.Tensor:
        _,_,height,width = pixel_values.shape
        
        patch_embeds = self.patch_embedding(pixel_values)
        
        embeddings = patch_embeds.flatten(2)
        
        embeddings = embeddings.transpose(1,2)
        
        embeddings = embeddings + self.position_embedding(self.position_ids)
        
        return embeddings
        

In [9]:
class SiglipAttention(nn.Module):
    
    def __init__(self,config):
        super().__init__()
        self.config = config
        self.embed_dim = config.hidden_size
        self.num_heads = config.num_attention_heads
        self.head_dim = self.embed_dim // self.num_heads
        self.scale = self.head_dim ** -0.5
        self.dropout = config.attention_dropout
        
        self.k_proj = nn.Linear(self.embed_dim,self.embed_dim)
        self.v_proj = nn.Linear(self.embed_dim,self.embed_dim)
        self.q_proj = nn.Linear(self.embed_dim,self.embed_dim)
        self.out_proj = nn.Linear(self.embed_dim,self.embed_dim)
    
    def forward(self,hidden_states: torch.Tensor) -> Tuple[torch.Tensor,
                                                          Optional[torch.Tensor]]:
        # hidden_states: [Batch_Size, Num_Patches, Embed_Dim]
        batch_size,seq_len,_ = hidden_states.size()
        # query_states: [Batch_Size, Num_Patches, Embed_Dim]

        query_states = self.q_proj(hidden_states)
        
        key_states = self.k_proj(hidden_states)
        
        values_states = self.v_proj(hidden_states)
        
        # query_states: [Batch_Size, Num_Heads, Num_Patches, Head_Dim]
        
        query_states = query_states.view(batch_size,seq_len,self.num_heads,self.head_dim).transpose(1,2)
        
        
        key_states = key_states.view(batch_size,seq_len,self.num_heads,self.head_dim).transpose(1,2)

        values_states = values_states.view(batch_size,seq_len,self.num_heads,self.head_dim).transpose(1,2)
        
        
        attn_weights = (torch.matmul(query_states,key_states.transpose(2,3))**self.scale)
        
        if attn_weights.size() != (batch_size, self.num_heads, seq_len, seq_len):
            raise ValueError(
                f"Attention weights should be of size {(batch_size, self.num_heads, seq_len, seq_len)}, but is"
                f" {attn_weights.size()}"
            )
        
        attn_weights = nn.functional.softmax(attn_weights,dim=-1,dtype=torch.float32).to(query_states.dtype)
        
        attn_weights = nn.functional.dropout(attn_weights,p=self.dropout,training=self.training)
        
        attn_output = torch.matmul(attn_weights,values_states)
        
        if attn_output.size() != (batch_size, self.num_heads, seq_len, self.head_dim):
            raise ValueError(
                f"`attn_output` should be of size {(batch_size, self.num_heads, seq_len, self.head_dim)}, but is"
                f" {attn_output.size()}"
            )
        
        
        attn_output = attn_output.transpose(1,2).contiguous()
        
        attn_output = attn_output.reshape(batch_size,seq_len,self.embed_dim)
        
        attn_output = self.out_proj(attn_output)
        
        return attn_output, attn_weights

In [1]:
!ls -a

.                              marian-finetuned-kde4-en-to-fr
..                             mmdl.ipynb
.git                           multi_modality.ipynb
.ipynb_checkpoints             simple_model_state_dict.pth
final_data.json                tokens.json
gpt_from_scratch.ipynb         translate_from_scratch.ipynb
hg_learn.ipynb


In [2]:
!pwd

/Users/liuchu/hg_learn
